# 특강 01: Spotify API 활용 및 실제 음원 특성 데이터 수집

## 학습 목표
- Spotify Web API 인증 및 기본 사용법을 익혀보자
- Track Features API를 활용한 음원 특성 데이터를 수집해보자
- API 응답 데이터를 파싱하여 데이터프레임으로 변환해보자
- API 사용량 제한 및 효율적인 데이터 수집 전략을 알아보자

In [ ]:
# 라이브러리 import
import pandas as pd
import requests
import json
import base64
from typing import Dict, List, Optional

print("🎵 특강 01: Spotify API 활용 및 실제 음원 특성 데이터 수집")
print("실제 음원 서비스의 데이터를 활용한 분석을 시작해보자!")

## 1. Spotify 앱 등록

### 1.1 Spotify Developer 계정 생성

실제 음원 데이터를 수집하기 위해 Spotify Web API를 사용해보자. 먼저 개발자 계정을 만들어야 한다.

1. [Spotify for Developers](https://developer.spotify.com/) 접속
2. 기존 Spotify 계정으로 로그인 (없다면 계정 생성)
3. "Dashboard"로 이동
4. "Create an App" 클릭

### 1.2 앱 등록 시 주의사항

⚠️ **중요: HTTPS 요구사항**

현재 Spotify는 보안상의 이유로 **모든 Redirect URI가 HTTPS를 사용해야 한다**.
- ✅ `https://localhost:8080/callback`
- ❌ `http://localhost:8080/callback` (더 이상 허용되지 않음)

따라서 로컬 개발 시에도 HTTPS 설정이 필요하거나, Client Credentials Flow를 사용해야 한다.

In [ ]:
# 앱 등록 정보 예시
print("📝 Spotify 앱 등록 정보 예시:")
print("App Name: Music Data Analysis")
print("App Description: Educational project for music data analysis")
print("Website: https://your-website.com (선택사항)")
print("Redirect URIs: https://localhost:8080/callback")
print("API/SDK: Web API 체크")
print()
print("등록 완료 후 받을 정보:")
print("- Client ID: 공개 식별자")
print("- Client Secret: 비밀 키 (절대 노출 금지!)")

## 2. 인증 방식 설명

Spotify API는 두 가지 주요 인증 방식을 제공한다:

### 2.1 Client Credentials Flow (추천)
- **용도**: 사용자 개인 정보가 필요 없는 공개 데이터 접근
- **장점**: 간단하고 HTTPS 요구사항 없음
- **단점**: 사용자별 플레이리스트 등 개인 데이터 접근 불가
- **적합한 경우**: 트랙 특성, 앨범 정보 등 공개 데이터 수집

### 2.2 Authorization Code Flow
- **용도**: 사용자 개인 데이터 접근 (플레이리스트, 재생 기록 등)
- **장점**: 모든 API 기능 사용 가능
- **단점**: HTTPS 필수, 복잡한 인증 과정
- **적합한 경우**: 개인화된 추천, 사용자 플레이리스트 분석

### 2.3 이 수업에서 사용할 방식

음원 특성 데이터 수집이 목적이므로 **Client Credentials Flow**를 사용하겠다.

In [ ]:
# SpotifyAPI 클래스 - Client Credentials Flow 구현
class SpotifyAPI:
    def __init__(self, client_id: str, client_secret: str):
        self.client_id = client_id
        self.client_secret = client_secret
        self.access_token = None
        
    def get_access_token(self) -> str:
        """Client Credentials Flow로 액세스 토큰 획득"""
        
        # 1. 클라이언트 자격 증명을 Base64로 인코딩
        credentials = f"{self.client_id}:{self.client_secret}"
        encoded_credentials = base64.b64encode(credentials.encode()).decode()
        
        # 2. 토큰 요청
        url = "https://accounts.spotify.com/api/token"
        headers = {
            "Authorization": f"Basic {encoded_credentials}",
            "Content-Type": "application/x-www-form-urlencoded"
        }
        data = {"grant_type": "client_credentials"}
        
        response = requests.post(url, headers=headers, data=data)
        
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data["access_token"]
            print(f"✅ 인증 성공! 토큰 유효시간: {token_data['expires_in']}초")
            return self.access_token
        else:
            print(f"❌ 인증 실패: {response.status_code}")
            print(response.text)
            return None
    
    def search_track(self, query: str, limit: int = 10) -> dict:
        """곡 검색"""
        if not self.access_token:
            print("먼저 get_access_token()을 호출하세요")
            return None
            
        url = "https://api.spotify.com/v1/search"
        headers = {"Authorization": f"Bearer {self.access_token}"}
        params = {
            "q": query,
            "type": "track",
            "limit": limit
        }
        
        response = requests.get(url, headers=headers, params=params)
        return response.json() if response.status_code == 200 else None
    
    def get_track_features(self, track_id: str) -> dict:
        """특정 곡의 음향 특성 데이터 가져오기"""
        if not self.access_token:
            print("먼저 get_access_token()을 호출하세요")
            return None
            
        url = f"https://api.spotify.com/v1/audio-features/{track_id}"
        headers = {"Authorization": f"Bearer {self.access_token}"}
        
        response = requests.get(url, headers=headers)
        return response.json() if response.status_code == 200 else None
    
    def get_multiple_track_features(self, track_ids: List[str]) -> dict:
        """여러 곡의 음향 특성 데이터 한번에 가져오기 (최대 100개)"""
        if not self.access_token:
            print("먼저 get_access_token()을 호출하세요")
            return None
            
        # 100개씩 나누어서 처리
        if len(track_ids) > 100:
            track_ids = track_ids[:100]
            print(f"⚠️  API 제한으로 처음 100개만 처리합니다")
        
        url = "https://api.spotify.com/v1/audio-features"
        headers = {"Authorization": f"Bearer {self.access_token}"}
        params = {"ids": ",".join(track_ids)}
        
        response = requests.get(url, headers=headers, params=params)
        return response.json() if response.status_code == 200 else None

print("🎵 Spotify API 클래스 완성!")
print("주요 메서드:")
print("- get_access_token(): 인증")
print("- search_track(): 곡 검색") 
print("- get_track_features(): 단일 곡 특성")
print("- get_multiple_track_features(): 다중 곡 특성")

## 3. API 호출 예시

이제 실제로 Spotify API를 사용해서 음원 데이터를 가져와 보자.

In [ ]:
# 실제 사용 예시 (데모 - 실제 API 키가 없어도 코드 구조 확인 가능)

# 1. API 객체 생성 및 인증
print("=== 1. 인증 과정 ===")
print("# 실제 사용 시:")
print("# spotify = SpotifyAPI('your_client_id', 'your_client_secret')")
print("# token = spotify.get_access_token()")
print()

# 2. 곡 검색 예시
print("=== 2. 곡 검색 예시 ===")
print("# BTS 곡 검색")
print("# search_result = spotify.search_track('BTS Dynamite', limit=5)")
print("# 예상 결과 구조:")
example_search = {
    "tracks": {
        "items": [
            {
                "id": "5QDLhrAOJJdNAmCTJ8xMyW",
                "name": "Dynamite",
                "artists": [{"name": "BTS"}],
                "popularity": 85
            }
        ]
    }
}
print(f"검색 결과 예시: {json.dumps(example_search, indent=2, ensure_ascii=False)}")
print()

# 3. 음향 특성 데이터 가져오기
print("=== 3. 음향 특성 데이터 ===")
print("# track_features = spotify.get_track_features('5QDLhrAOJJdNAmCTJ8xMyW')")
print("# 예상 음향 특성 데이터:")
example_features = {
    "acousticness": 0.00242,
    "danceability": 0.746,
    "energy": 0.765,
    "instrumentalness": 0.000001,
    "liveness": 0.0817,
    "loudness": -6.166,
    "speechiness": 0.0818,
    "tempo": 114.054,
    "valence": 0.548
}
print(json.dumps(example_features, indent=2))

In [ ]:
# 데이터프레임 변환 함수
def create_tracks_dataframe(search_results: dict, features_data: dict) -> pd.DataFrame:
    """검색 결과와 음향 특성을 결합하여 데이터프레임 생성"""
    
    # 검색 결과에서 기본 정보 추출
    tracks_info = []
    for track in search_results['tracks']['items']:
        tracks_info.append({
            'track_id': track['id'],
            'name': track['name'],
            'artist': ', '.join([artist['name'] for artist in track['artists']]),
            'popularity': track['popularity']
        })
    
    # 데이터프레임 생성
    df = pd.DataFrame(tracks_info)
    
    # 음향 특성 데이터 결합
    if features_data and 'audio_features' in features_data:
        features_df = pd.DataFrame(features_data['audio_features'])
        # track_id를 기준으로 병합
        df = df.merge(features_df, left_on='track_id', right_on='id', how='left')
        # 불필요한 컬럼 제거
        df = df.drop(['id', 'uri', 'track_href', 'analysis_url', 'type'], axis=1, errors='ignore')
    
    return df

# 사용 예시 (데모 데이터)
print("📊 데이터프레임 변환 예시:")

# 가상의 검색 결과와 음향 특성 데이터로 데모 데이터프레임 생성
demo_search = {
    "tracks": {
        "items": [
            {"id": "track1", "name": "Dynamite", "artists": [{"name": "BTS"}], "popularity": 85},
            {"id": "track2", "name": "Butter", "artists": [{"name": "BTS"}], "popularity": 90}
        ]
    }
}

demo_features = {
    "audio_features": [
        {"id": "track1", "danceability": 0.746, "energy": 0.765, "valence": 0.548, "tempo": 114.054},
        {"id": "track2", "danceability": 0.896, "energy": 0.774, "valence": 0.978, "tempo": 110.030}
    ]
}

demo_df = create_tracks_dataframe(demo_search, demo_features)
print("생성된 데이터프레임:")
print(demo_df.head())
print(f"\n데이터프레임 형태: {demo_df.shape}")
print(f"컬럼: {list(demo_df.columns)}")

## 4. 주의사항 및 팁

**API 사용량 제한:**
- Spotify API는 시간당 요청 수 제한이 있다
- 너무 많은 요청을 보내면 일시적으로 차단될 수 있다
- `time.sleep(0.1)` 등으로 요청 간격을 두는 것이 좋다

**데이터 저장:**
- 수집한 데이터는 CSV 파일로 저장해두자
- API 호출 결과를 재사용할 수 있어 효율적이다

**보안:**
- Client Secret은 절대 공개하지 말자
- 환경 변수나 별도 설정 파일을 사용하는 것이 안전하다